In [1]:
from random import random
import numpy as np
import time
import math
import random
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import glob
from IPython.display import Image
from IPython.display import HTML

In [2]:
def read_tsp_file(nodes_file):
    infile = open(nodes_file, 'r')
    content = infile.readline().strip().split()
    print("File Name: ", content[2])

    while content[0] != 'Node_Co-Ordinate_Section':
        if(content[0] == 'Dimension'):
            dimension = content[2]
        content = infile.readline().strip().split()
    node_list = []
    place_list = []
    print('Dimension', dimension)
    N = int(dimension)
    for i in range(0, N):
        x, y, z = infile.readline().strip().split()[:]
        node_list.append([float(y), float(z)])
        place_list.append(x)

    # Close input file
    infile.close()
    return node_list, place_list


In [3]:
class SimAnneal():
    def __init__(self, coordinates, place, stopping_iter, N=-1, nodes=-1, temp=-1, stopping_temperature=-1):
        self.coords = coordinates
        self.place = place
        self.Num = len(coordinates)
        self.stopping_temperature = 1e-8
        self.temp = 1000
        self.stopping_iter = stopping_iter
        self.iteration = 1
        self.nodes = [i for i in range(self.Num)]
        self.best_path = None
        self.best_cost = float("Inf")
        self.cost_list = []
        self.path_history = []

# Total cost of the current path.
    def calculated_path_cost(self, solution):
        cost = 0
        for i in range(self.Num):
            cost += self.dist(solution[i % self.Num], solution[(i + 1) % self.Num])
        return cost


# Euclidean distance between two nodes.

    def dist(self, node0, node1):
        coord0, coord1 = self.coords[node0], self.coords[node1]
        distance = math.sqrt(
            ((coord0[0] - coord1[0]) ** 2) + ((coord0[1] - coord1[1]) ** 2))
        return distance

    def accept(self, candidate):
        candidate_cost = self.calculated_path_cost(candidate)

# Accept with probability 1 if candidate is better then currrent

        if candidate_cost < self.best_cost:
            self.best_cost = candidate_cost
            self.best_path = candidate

        else:
            # Probabily of accepting if candidate is worst than current.
            #  Prabability = 𝑒^(-𝛥𝐸/T)
            probability_accept = math.exp(-abs(candidate_cost -
                                               self.current_cost)/self.temp)
            if random.random() < probability_accept:
                self.current_cost = candidate_cost
                self.current_path = candidate


# Greedy solution(nearest neighbuor)


    def intial_solution(self):
        current_node = random.choice(self.nodes)
        path = [current_node]
        remaining_node = set(self.nodes)
        remaining_node.remove(current_node)
        while remaining_node:
            next_node = min(
                remaining_node, key=lambda x: self.dist(current_node, x))
            current_node = next_node
            remaining_node.remove(current_node)
            path.append(current_node)
        initial_cost = self.calculated_path_cost(path)
        if(self.best_cost > initial_cost):
            self.best_cost = initial_cost
            self.best_path = path
        self.cost_list.append(initial_cost)
        self.path_history.append(path)
        return path, initial_cost

# Simulated annealing algorithm
    def simulated_annealing(self):
        self.current_path, self.current_cost = self.intial_solution()
        while self.temp >= self.stopping_temperature and self.iteration < self.stopping_iter:
            candidate = list(self.current_path)
            l = random.randint(2, self.Num - 1)
            i = random.randint(0, self.Num - 1)
            candidate[i:(i+l)] = reversed(candidate[i:(i+l)])
            self.accept(candidate)

#           taking alpha = 0.9995
            self.temp *= 0.9995
            self.iteration += 1
            self.cost_list.append(self.current_cost)
            self.path_history.append(self.current_path)
        print("Best cost obtained:", self.best_cost)

    def display_optimal_path(self):
        n = len(self.best_path)
        tour = ''
        for i in range(0, (n)):
            x = self.best_path[i]
            tour = tour + self.place[x]+' -> '
        tour += self.place[self.best_path[0]]
        print("Optimal Path :", tour)

# Animated visualization of TSP
    def animateTSP(self):
      key_frames_mult = len(self.path_history) // 1500
      fig, ax = plt.subplots()
      line, = plt.plot([], [], lw=2)

      def init():
        x = [self.coords[i][0] for i in self.path_history[0]]
        y = [self.coords[i][1] for i in self.path_history[0]]
        plt.plot(x, y, 'co')

        extra_x = (max(x) - min(x)) * 0.05
        extra_y = (max(y) - min(y)) * 0.05
        ax.set_xlim(min(x) - extra_x, max(x) + extra_x)
        ax.set_ylim(min(y) - extra_y, max(y) + extra_y)

        line.set_data([], [])
        return line,

      def update(frame):
        x = [self.coords[i, 0] for i in self.path_history[frame] + [self.path_history[frame][0]]]
        y = [self.coords[i, 1] for i in self.path_history[frame] + [self.path_history[frame][0]]]
        line.set_data(x, y)
        return line

      ani = FuncAnimation(fig, update, frames=range(0, len(self.path_history), key_frames_mult), init_func=init, interval=3, repeat=False)
      display(HTML(ani.to_jshtml()))

   
#  Plot the fitness through iterations.
    def plot_learning(self):
        initial_cost = self.cost_list[0]
        plt.plot([i for i in range(len(self.cost_list))], self.cost_list)
        line_init = plt.axhline(y=initial_cost, color='r', linestyle='--')
        line_min = plt.axhline(y=self.best_cost, color='g', linestyle='--')
        plt.title("Learning progress")
        plt.legend([line_init, line_min], ['Initial Cost', 'Optimized Cost'])
        plt.ylabel("Cost")
        plt.xlabel("Iteration")
        plt.show()

In [ ]:
def main():
    # generate_random_coords(100)
    nodes, place = tsp_read("Data/rajasthan.tsp")
    coords = np.array(nodes)
    n = len(coords)
    start = time.time_ns()
    sa = SimAnneal(coords, place, stopping_iter=n*10000000000000000)
    end = time.time_ns()
    print('Execution Time', end-start)
    sa.simulated_annealing()
    sa.display_optimal_path()
    # sa.animateTSP()
    sa.plot_learning()


if __name__ == "__main__":
    main()